In [ ]:
def filter_generator_node(state: MainState) -> MainState:
    try:
        query = state["user_query"]
        
        prompt = (
            "Based on the user query, analyze and choose exactly ONE category from this list:\n"
            "- research\n"
            "- report\n"  
            "- conversation\n"
            "- article\n\n"
            f"Query: {query}\n\n"
            "Respond with only ONE word from the list above. No explanations, no other text."
        )
        
        llm_response = llm_model.with_structured_output(FilterCategory).invoke(prompt)
        filter_word = llm_response.category.lower()
        
        # Validate response
        allowed_categories = ["research", "report", "conversation", "article"]
        
        if filter_word in allowed_categories:
            print(f"filter generated: {filter_word}")
            return {"label": filter_word}
        else:
            print(f"invalid filter: {filter_word}, using fallback")
            return {"label": "conversation"}  # Safe fallback
            
    except Exception as e:
        print(f"filter generation failed: {e}")
        return {"label": "conversation"}  # Safe fallback

In [ ]:
def semantic_search_a_node(state: MainState) -> MainState:
    try:
        query = state["user_query"]
        filter_value = state.get("label")  # Fixed field name
        
        filters = {"category": [filter_value]}
        semantic_result = semantic_search_a(query, top_k=10, filters=filters, threshold=0.3)
        
        result = [{'id': item['id'], 'text': item['text']} for item in semantic_result]
        
        if result:
            print(f"found {len(result)} results for {filter_value}")
        else:
            print("no search results found")
            
        return {"semantic_search_a_results": result}
        
    except Exception as e:
        print(f"search failed: {e}")
        return {"semantic_search_a_results": []}

In [ ]:
def filter_document_node(state: MainState) -> MainState:
    try:
        graded_documents = state.get("graded_documents", [])
        
        if not graded_documents:
            print("no documents to filter")
            return {"semantic_search_a_results": []}
        
        # Filter documents by threshold (built-in value)
        threshold = 75
        filtered_docs = []
        
        for doc in graded_documents:
            if doc.get("grade", 0) >= threshold:
                # Keep only id and text for final output
                filtered_docs.append({
                    "id": doc["id"],
                    "text": doc["text"]
                })
        
        print(f"filtered {len(filtered_docs)} from {len(graded_documents)} documents")
        
        return {"semantic_search_a_results": filtered_docs}
        
    except Exception as e:
        print(f"filtering failed: {e}")
        return {"semantic_search_a_results": []}